In [1]:
import time
import pywikibot
import mwparserfromhell
import csv
import json
import re
import xml.sax
from bs4 import BeautifulSoup
import requests

In [2]:
site = pywikibot.Site("id", "wikipedia")
art = pywikibot.Page(site, 'Raden Mattaher')


wikicode = mwparserfromhell.parse(art.get())
matches = wikicode.filter_templates(matches = "Infobox")
properties = {param.name.strip(): param.value
                                for param in matches[0].params}
print(properties)

{'name': ' Raden Mattaher\n', 'image': ' Raden Mattaher.jpg\n', 'imagesize': ' 200px\n', 'alt': ' \n', 'caption': ' \n', 'birth_name': ' \n', 'birth_date': ' 1871\n', 'birth_place': ' Dusun Sekamis, [[Kesultanan Jambi]]\n', 'death_date': ' {{death date and age|1907|9|10|1871|df=y}}\n', 'death_place': ' Dusun Muaro Jambi, [[Keresidenan Jambi]], [[Hindia Belanda]]\n', 'other_names': ' Raden Mohammad Tahir, Pangeran Mattahir\n', 'known_for': " Pejuang Kemerdekaan [[Indonesia]] dari [[Jambi]], dan [[Pahlawan Nasional Indonesia]]<ref>[https://m.republika.co.id/amp/qjk8kb335 Raden Mattaher, Singo Kumpeh yang Jadi Pahlawan Nasional] ''republika'' 10 November 2020</ref>\n", 'occupation': ' \n', 'religion': ' \n', 'spouse': ' \n', 'children': ' \n', 'parents': ' Raden Kusin (ayah)<br> Ratumas Esa (ibu)\n', 'relatives': ' [[Sultan Thaha Syaifuddin]] (kakek)\n'}


In [62]:
print(matches)

["{{Infobox person\n| name               = Raden Mattaher\n| image              = Raden Mattaher.jpg\n| imagesize          = 200px\n| alt                = \n| caption            = \n| birth_name         = \n| birth_date         = 1871\n| birth_place        = Dusun Sekamis, [[Kesultanan Jambi]]\n| death_date         = {{death date and age|1907|9|10|1871|df=y}}\n| death_place        = Dusun Muaro Jambi, [[Keresidenan Jambi]], [[Hindia Belanda]]\n| other_names        = Raden Mohammad Tahir, Pangeran Mattahir\n| known_for          = Pejuang Kemerdekaan [[Indonesia]] dari [[Jambi]], dan [[Pahlawan Nasional Indonesia]]<ref>[https://m.republika.co.id/amp/qjk8kb335 Raden Mattaher, Singo Kumpeh yang Jadi Pahlawan Nasional] ''republika'' 10 November 2020</ref>\n| occupation         = \n| religion           = \n| spouse             = \n| children           = \n| parents            = Raden Kusin (ayah)<br> Ratumas Esa (ibu)\n| relatives          = [[Sultan Thaha Syaifuddin]] (kakek)\n}}"]


In [59]:
for key,value in properties.items():
    print(key,":",value)
    print("===========")

name :  Sultan Thaha Syaifuddin

title :  

titletext :  

image :  Sultan Thaha Syaifuddin.jpg

caption :  

succession :  [[Kesultanan Jambi#Daftar Penguasa|Sultan Jambi]] ke-20

reign :  1855 - 26 April 1904

date1 :  

date2 :  

reign-type :  

coronation :  

predecessor :  Sultan Abdurrahman Nazaruddin

successor :  

royal house :  

father :  Sultan Muhammad Fachruddin

issue :  

birth_date :  {{birth year|1816}}

birth_place :  [[Kota Jambi|Tanah Pilih]], [[Kesultanan Jambi]]

death_date :  {{death date|1904|4|26}} (87 atau 88 tahun)

death_place :  Betung Bedarah, [[Tebo Ilir, Tebo|Tebo Ilir]], [[Tebo]]



In [103]:
wikicode = mwparserfromhell.parse(properties['commander1'])
print(str(properties['commander1']))
temp = re.sub("<br>","{{br}}",str(properties['commander1']))
print(re.sub("{{br}}","\n",temp))

 {{flagicon|Belanda}} [[Johan Harmen Rudolf Köhler|Johan Köhler]]{{KIA}}
<br>{{flagicon|Belanda}} [[Jan van Swieten]] 
<br>{{flagicon|Belanda}} [[Johannes Ludovicius Jakobus Hubertus Pel|Johannes Pel]]{{KIA}}
<br>{{flagicon|Belanda}} [[Karel van der Heijden]]{{WIA}}
<br>{{flagicon|Belanda}} [[Henry Demmeni]]{{DOW}}
<br>{{flagicon|Belanda}} [[Jan Jacob Karel de Moulin]]{{KIA}}
<br>{{flagicon|Belanda}} [[G.C.E. van Daalen (1863-1930)|Gotfried van Daalen]]
<br>{{flagicon|Belanda}} [[Johan Cornelis van der Wijck]]
{{flagicon|Belanda}} [[Johannes Benedictus van Heutsz]]

 {{flagicon|Belanda}} [[Johan Harmen Rudolf Köhler|Johan Köhler]]{{KIA}}

{{flagicon|Belanda}} [[Jan van Swieten]] 

{{flagicon|Belanda}} [[Johannes Ludovicius Jakobus Hubertus Pel|Johannes Pel]]{{KIA}}

{{flagicon|Belanda}} [[Karel van der Heijden]]{{WIA}}

{{flagicon|Belanda}} [[Henry Demmeni]]{{DOW}}

{{flagicon|Belanda}} [[Jan Jacob Karel de Moulin]]{{KIA}}

{{flagicon|Belanda}} [[G.C.E. van Daalen (1863-1930)|Gotfried 

# military conflict

In [2]:
class Infobox():
    def __init__(self,title):
        self.title = title
        site = pywikibot.Site("id", "wikipedia")
        art = pywikibot.Page(site, self.title)
    

        wikicode = mwparserfromhell.parse(art.get())
        matches = wikicode.filter_templates(matches = self.type)
        self.infobox = {param.name.strip(): re.sub("\[\[File:[^\]]*\]\]","",re.sub("\[\[Berkas:[^\]]*\]\]","",str(param.value)))
                            for param in matches[0].params}
#         print(self.infobox)
        self.clean_infobox = self.clean_data()
        self.clean_infobox['wikititle'] = title
    def clean_link(self,key):
        return self.filter_wikilinks(self.infobox.get(key,""))
    def clean_text(self,key):
        return self.text(self.infobox.get(key,""))
    def clean_list(self,key):
        return self.list(self.infobox.get(key,""))
    def clean_list2(self,key):
        return self.list2(self.infobox.get(key,""))
    def clean_br(self,key):
        temp = self.infobox.get(key,"")
        temp = re.sub("{{br}}","<br>",str(self.infobox.get(key,"")))
        temp = re.sub("<br />","<br>",temp)
        temp = re.sub("<br/>","<br>",temp)
        temp = re.sub("<br>","\n",temp)
        temp = temp.split("\n")
        res = []
        for i in temp:
            if i != "" :
                res.append(self.text(i.strip()))
        return res
    def clean_date_template(self,key):
        wikitext = self.infobox.get(key,"")
        temp = self.clean_birth_date(wikitext)
        if(temp != ""):
            return temp
        temp = self.clean_death_date_and_age(wikitext)
        if(temp != ""):
            return temp
        return self.clean_template(wikitext)
    def clean_date(self,key):
        data = self.infobox.get(key,"")
        temp = re.split("[-|\u2013]",data)
        print(temp)
        res={}
        if len(temp) > 0:
            res['start'] = self.text(temp[0].strip())
        if len(temp) > 1:
            res['end'] = self.text(temp[1].strip())
        return res
    def clean_template(self,key):
        wikitext = self.infobox.get(key,"")
        wikicode = mwparserfromhell.parse(wikitext)
        return re.sub("\n","",wikicode.strip_code(keep_template_params=True).strip())
    def template(self,wikitext):
        wikicode = mwparserfromhell.parse(wikitext)
        return re.sub("\n","",wikicode.strip_code(keep_template_params=True).strip())
    def strip_template(self,wikitext):
        wikicode = mwparserfromhell.parse(wikitext)
        return re.sub("\n","",wikicode.strip_code(keep_template_params=True).strip())
    def filter_wikilinks(self,wikitext):
        wikicode = mwparserfromhell.parse(wikitext)
        link = wikicode.filter_wikilinks()
        temp = []
        for i in link:
            temp.append(str(i.title))
        return temp
    def text(self,wikitext):
        wikicode = mwparserfromhell.parse(wikitext)
        return re.sub("\n","",wikicode.strip_code().strip())
    def list(self,wikitext):
        wikicode = mwparserfromhell.parse(wikitext)
        template = wikicode.filter_templates(matches="ubl")
        temp = []
        if(len(template) > 0):
            for i in template[0].params :
                temp.append(str(self.text(i.value)))
        template = wikicode.filter_templates(matches="bulleted list")
        if(len(template) > 0):
            for i in template[0].params :
                temp.append(str(self.text(i.value)))
        template = wikicode.filter_templates(matches="plainlist")
        if(len(template) > 0):
            for i in str(template[0].params[0].value).split("*") :
                temp.append(str(self.text(i)))
        return temp
    def clean_marriage(self,key):
        wikitext = self.infobox.get(key,"")
        wikicode = mwparserfromhell.parse(wikitext)
        template = wikicode.filter_templates(matches="marriage")
        temp = []
        print(template)
        if(len(template) > 0):
            for i in template :
                temp.append(str(self.strip_template(i.params[0].value)))
        return temp
    def clean_birth_date(self,wikitext):
        wikicode = mwparserfromhell.parse(wikitext)
        template = wikicode.filter_templates(matches="birth date")
        if(len(template) >0):
            date = ['January','February','March','April','May','June','July','August','September','October','November','December']
            temp = re.findall("\d+",self.template(wikitext))
            print(wikitext)
            print(temp)
            temp[1] = date[int(temp[1])-1]
            return "-".join(temp[::-1])
        return ""
    def clean_death_date_and_age(self,wikitext):
        wikicode = mwparserfromhell.parse(wikitext)
        template = wikicode.filter_templates(matches="death date and age")
        if(len(template) >0):
            date = ['January','February','March','April','May','June','July','August','September','October','November','December']
            temp = re.findall("\d+",self.template(wikitext))
            print(wikitext)
            print(temp)
            temp[1] = date[int(temp[1])-1]
            return "-".join(temp[:3][::-1])
        return ""
    def list2(self,wikitext):
        wikicode = mwparserfromhell.parse(wikitext)
        template = wikicode.filter_templates(matches="ubl")
        temp = []
        if(len(template) > 0):
            for i in template[0].params :
                temp.append(str(self.strip_template(i.value)))
        template = wikicode.filter_templates(matches="bulleted list")
        if(len(template) > 0):
            for i in template[0].params :
                temp.append(str(self.strip_template(i.value)))
        template = wikicode.filter_templates(matches="plainlist")
        if(len(template) > 0):
            if(template[0].params[0].value!= "nowrap"):
                for i in str(template[0].params[0].value).split("*") :
                    temp.append(str(self.strip_template(i)))
            else :
                for i in str(template[0].params[1].value).split("*") :
                    temp.append(str(self.strip_template(i)))
        return temp
    

In [35]:
class Infobox_military_conflict(Infobox):
    def __init__(self,title):
        self.type = "Infobox Military Conflict"
        Infobox.__init__(self,title)
    def clean_data(self):
        temp = {}
        temp['conflict'] = self.clean_text('conflict')
        temp['partof'] = self.clean_text('partof')
        temp['image'] = self.clean_text('image')
        temp['caption'] = self.clean_text('caption')
        temp['date'] = self.clean_date('date')
        temp['place'] = self.clean_link('place')
        temp['casus'] = self.clean_text('casus')
        temp['teritory'] = self.clean_text('teritory')
        temp['result'] = self.clean_text('result')
        temp['combatant1'] = self.clean_list("combatant1")
        if(len(temp['combatant1']) == 0):
            temp['combatant1'] = self.clean_br('combatant1')
        temp['combatant2'] = self.clean_list("combatant2")
        if(len(temp['combatant2']) == 0):
            temp['combatant2'] = self.clean_br('combatant2')
        temp['commander1'] = self.clean_list("commander1")
        if(len(temp['commander1']) == 0):
            temp['commander1'] = self.clean_br('commander1')
        temp['commander2'] = self.clean_list("commander2")
        if(len(temp['commander2']) == 0):
            temp['commander2'] = self.clean_br('commander2')
        temp['units1'] = self.clean_list("units1")
        if(len(temp['units1']) == 0):
            temp['units1'] = self.clean_br('units1')
        temp['units2'] = self.clean_list("units2")
        if(len(temp['units2']) == 0):
            temp['units2'] = self.clean_br('units2')
        temp['strength1'] = self.clean_text('strength1')
        temp['strength2'] = self.clean_text('strength1')
        temp['casualties1'] = self.clean_text('casualties1')
        temp['casualties2'] = self.clean_text('casualties2')
        temp['notes'] = self.clean_text('notes')
        return temp
    

In [58]:
diponegoro = Infobox_military_conflict("Perang Diponegoro")
print(diponegoro.clean_infobox)

{'conflict': 'Perang Diponegoro', 'partof': '', 'image': '300px', 'caption': 'Lukisan Peristiwa Penangkapan Pangeran Diponegoro oleh Raden Saleh', 'date': '21 Juli 1825 - 9 Februari 1830 1840', 'place': '[[Yogyakarta]],[[Surakarta]],[[Jawa Tengah]],[[Jawa Timur]]', 'casus': 'Jalan yang dibangun Belanda melintasi makam leluhur Pangeran Diponegoro 6', 'teritory': '', 'result': 'Kemenangan BelandaPenangkapan dan pengasingan Pangeran Diponegoro ke MakassarToby Alice Volkman: Sulawesi: island crossroads of Indonesia, Passport Books, 1990, ISBN 0844299065,  page 73.', 'combatant1': '[[Imperium Belanda|Belanda]],[[Kesultanan Yogyakarta]],[[Pasukan Tulungan]]', 'combatant2': 'Pasukan Jawa/Diponegoro', 'commander1': '[[Hendrik Merkus de Kock|Hendrik M. de Kock]],[[Hermanus W. Dotulong]]', 'commander2': '[[Diponegoro]],[[Sentot Prawirodirdjo]],[[Kiai Madja]],[[Nyi Ageng Serang]],[[Pakubuwana VI]],[[Tumenggung Prawirodigdoyo]]', 'strength1': '50.000', 'strength2': '50.000', 'casualties1': '15.000

In [36]:
infobox = json.load(open("infobox\\json\\v1Infobox_Military_Conflict1.json"))
print(len(infobox))
daftar_infobox = []
for i in infobox :
    daftar_infobox.append(Infobox_military_conflict(i['entity']))
print(len(daftar_infobox))

11
['21 Juli [[1825]] ', ' 9 Februari [[1830 1840]]\n']
[' 1803', '1837\n']
[' 1873', '1904\n']
[' 15', '16 Juni 1908\n']
['22 Januari 1293', 'awal Agustus 1293{{sfn', 'Hung', '2022', 'p=7}} (8 bulan)\n']
[' 15', '19 Oktober 1945\n']
['  5 Agustus 1771 ', ' 11 Oktober 1772\n']
[' 19', '20 Juni 1812\n']
[' 20 November', '17 Desember [[1810]]\n']
['1641\n']
[' 1873', '1904\n']
11


In [38]:
key = 'date'
for i in daftar_infobox :
    print(i.infobox.get(key,""))
    print("-----------------")
    print(i.clean_infobox[key])
#     for j,k in i.clean_infobox.items():
#         print(j,":",k)
    print("################")

21 Juli [[1825]] - 9 Februari [[1830 1840]]

-----------------
{'start': '21 Juli 1825', 'end': '9 Februari 1830 1840'}
################
 1803–1837

-----------------
{'start': '1803', 'end': '1837'}
################
 1873–1904

-----------------
{'start': '1873', 'end': '1904'}
################
 15-16 Juni 1908

-----------------
{'start': '15', 'end': '16 Juni 1908'}
################
22 Januari 1293–awal Agustus 1293{{sfn|Hung|2022|p=7}} (8 bulan)

-----------------
{'start': '22 Januari 1293', 'end': 'awal Agustus 1293{{sfn'}
################
 15–19 Oktober 1945

-----------------
{'start': '15', 'end': '19 Oktober 1945'}
################
  5 Agustus 1771 - 11 Oktober 1772

-----------------
{'start': '5 Agustus 1771', 'end': '11 Oktober 1772'}
################
 19-20 Juni 1812

-----------------
{'start': '19', 'end': '20 Juni 1812'}
################
 20 November-17 Desember [[1810]]

-----------------
{'start': '20 November', 'end': '17 Desember 1810'}
################
1641

-----

In [25]:
a = "15-16 Juni 1908"
print(a.split("-"))

['15', '16 Juni 1908']


In [5]:
for i in daftar_infobox :
#     print(i.infobox['place'])
#     print("-----------------")
    for j,k in i.clean_infobox.items():
        print(j,":",k)
    print("################")

conflict : Perang Diponegoro
partof : 
image : 
caption : Lukisan Peristiwa Penangkapan Pangeran Diponegoro oleh Raden Saleh
date : 21 Juli 1825 - 9 Februari 1830 1840
place : ['Yogyakarta', 'Surakarta', 'Jawa Tengah', 'Jawa Timur']
casus : Jalan yang dibangun Belanda melintasi makam leluhur Pangeran Diponegoro 6
teritory : 
result : Kemenangan BelandaPenangkapan dan pengasingan Pangeran Diponegoro ke MakassarToby Alice Volkman: Sulawesi: island crossroads of Indonesia, Passport Books, 1990, ISBN 0844299065,  page 73.
combatant1 : ['Belanda', 'Kesultanan Yogyakarta', 'Pasukan Tulungan']
combatant2 : ['Pasukan Jawa/Diponegoro']
commander1 : ['Hendrik M. de Kock', 'Hermanus W. Dotulong']
commander2 : ['Diponegoro', 'Sentot Prawirodirdjo', 'Kiai Madja', 'Nyi Ageng Serang', 'Pakubuwana VI', 'Tumenggung Prawirodigdoyo']
units1 : []
units2 : []
strength1 : 50.000
strength2 : 50.000
casualties1 : 15.000
casualties2 : Serdadu Jawa:20,000 tewas dalam perangClodfelter, Michael, Warfare and Armed

In [148]:
string = "{{flagicon|Belanda}} [[Johannes Ludovicius Jakobus Hubertus Pel|Johannes Pel]]{{KIA}}"
print(re.sub("\{\{[^\}]*\}\}","",string))

['{{flagicon|Belanda}}', '{{KIA}}']


In [7]:
print(type([1,2,3]))

<class 'list'>


In [11]:
def to_csv(data,path,root):
    exist = []
    simple = []
    for info in data:
        for key,value in info.clean_infobox.items():
#             print(type(value))
            if(type(value) is list ):
                with open("infobox\\"+path+key+".csv",'a',encoding='utf-8') as file:
                    writer = csv.writer(file)
                    if not (key in exist):
                        writer.writerow(['key',key])
                        exist.append(key)
                    for i in value:
                        writer.writerow([info.clean_infobox[root],i])
            elif(type(value) is dict):
                with open("infobox\\"+path+key+"-date.csv",'a',encoding='utf-8') as file:
                    writer = csv.writer(file)
                    if not (key in exist):
                        writer.writerow(['key',"start-"+key,"end-"+key])
                        exist.append(key)
                    writer.writerow([info.clean_infobox[root],value['start'],value.get('end',"")])
            else:
                if not(key in simple):
                    simple.append(key)
    with open("infobox\\"+path+"simple.csv",'w',encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(simple)
        print(simple)
        for info in data:
            temp = []
            for i in simple:
                temp.append(info.clean_infobox[i])
            writer.writerow(temp)
            
                

                


In [10]:
to_csv(daftar_infobox,"military_conflict\\",'conflict')

NameError: name 'to_csv' is not defined

In [182]:
with open("infobox\\clean.json",'w') as file :
    out = []
    for i in daftar_infobox :
        out.append(i.clean_infobox)
    file.write(json.dumps(out))

# Infobox person

In [88]:
class Infobox_person(Infobox):
    def __init__(self,title):
        self.type = "Infobox person"
        Infobox.__init__(self,title)
    def clean_data(self):
        temp = {}
        temp['name'] = self.clean_text('name')
        temp['image'] = self.clean_text('image')
        temp['birth_place'] = self.clean_link('birth_place')
        temp['death_place'] = self.clean_link('death_place')
        temp['death_cause'] = self.clean_text('death_cause')
#         temp['resting_place_coordinates'] = self.clean_text('resting_place_coordinates')
        temp['monuments'] = self.clean_text('monuments')
        temp['ethnicity'] = self.clean_text('ethnicity')
        temp['occupation'] = self.clean_text('occupation')
        temp['religion'] = self.clean_text("religion")
        temp['denomination'] = self.clean_text("denomination")
        temp['parents'] = self.clean_br("parents")
        temp['awards'] = self.clean_text("awards")
        temp['caption'] = self.clean_text("caption")
        temp['nationality'] = self.clean_text("nationality")
        temp['spouse'] = self.clean_text('spouse')
        temp['birth_date'] = self.clean_text('birth_date')
        if temp['birth_date'] == "":
            temp['birth_date'] = self.clean_template('birth_date')
        temp['death_date'] = self.clean_text('death_date')
        if temp['death_date'] == "":
            temp['death_date'] = self.clean_template('death_date')
        temp['other_names'] = self.clean_text('other_names')
        temp['known_for'] = self.clean_text('known_for')
        temp['burial_place'] = self.clean_text('burial_place')
        temp['pre-nominals'] = self.clean_text('pre-nominals')
        temp['birth_name'] = self.clean_text('birth_name')
        temp['birthname'] = self.clean_text('birthname')
        temp['restingplace'] = self.clean_link('restingplace')
        temp['yearsactive'] = self.clean_text('yearsactive')
        temp['years_active'] = self.clean_text('years_active')
        temp['resting_place'] = self.clean_text('resting_place')
        
        temp['spouse-type'] = self.clean_text('spouse-type')
#         temp['module'] = self.clean_text('module')
        temp['title'] = self.clean_text('title')
        temp['father'] = self.clean_text('father')
        temp['house'] = self.clean_text('house')
        temp['partner'] = self.clean_text('partner')
        temp['mother'] = self.clean_text('mother')
        temp['website'] = self.clean_text('website')
        temp['relatives'] = self.clean_text('relatives')
        return temp

In [89]:
infobox = json.load(open("infobox\\json\\v1Infobox_person24.json"))
print(len(infobox))
daftar_infobox = []
for i in infobox :
    daftar_infobox.append(Infobox_person(i['entity']))
print(len(daftar_infobox))

11
11


In [82]:
property = set()
for i in daftar_infobox:
    for key,value in i.infobox.items():
        if re.sub("\s","",str(value)) != "" :
            property.add(key)
print(len(property))

for i in property :
    print(i)

41
birth_name
spouse-type
ethnicity
religion
image
alt
image_size
module
burial_place
birth_date
birth_place
pre-nominals
nationality
death_place
birthname
title
imagesize
other_names
father
death_cause
occupation
years_active
caption
restingplace
house
partner
awards
parents
name
spouse
denomination
known_for
monuments
mother
yearsactive
website
relatives
death_date
resting_place_coordinates
signature_type
resting_place


In [101]:
for i in daftar_infobox :
    print(i.infobox.get('signature_type',""))
    print("-----------------")
    print(i.clean_infobox['signature_type'])
#     for j,k in i.clean_infobox.items():
#         print(j,":",k)
    print("################")


-----------------

################

-----------------

################

-----------------

################

-----------------

################

-----------------

################
 Tanda tangan

-----------------
Tanda tangan
################

-----------------

################

-----------------

################

-----------------

################

-----------------

################

-----------------

################


# infobox office holder

In [3]:
def get_property(source,type):
    infobox = json.load(open(source))
    res = set()
    for i in infobox :
        title = i['entity']
        site = pywikibot.Site("id", "wikipedia")
        art = pywikibot.Page(site, title)
    

        wikicode = mwparserfromhell.parse(art.get())
        matches = wikicode.filter_templates(matches = type)
        if(len(matches) > 0):
            property = {param.name.strip(): re.sub("\[\[File:[^\]]*\]\]","",re.sub("\[\[Berkas:[^\]]*\]\]","",str(param.value)))
                                for param in matches[0].params}
            for key,value in property.items():
                if re.sub("\s","",str(value)) != "" :
                    res.add(key)

    print(len(res))

    for i in res :
        print(i)
    return res

In [5]:
get_property("infobox\\json\\v1Infobox_person24.json","Infobox person")

41
image_size
website
birth_name
death_place
caption
birthname
parents
restingplace
burial_place
image
partner
spouse-type
spouse
resting_place
known_for
father
signature_type
module
title
denomination
awards
ethnicity
resting_place_coordinates
monuments
other_names
mother
occupation
death_cause
birth_place
birth_date
death_date
nationality
religion
relatives
name
pre-nominals
imagesize
yearsactive
alt
years_active
house


{'alt',
 'awards',
 'birth_date',
 'birth_name',
 'birth_place',
 'birthname',
 'burial_place',
 'caption',
 'death_cause',
 'death_date',
 'death_place',
 'denomination',
 'ethnicity',
 'father',
 'house',
 'image',
 'image_size',
 'imagesize',
 'known_for',
 'module',
 'monuments',
 'mother',
 'name',
 'nationality',
 'occupation',
 'other_names',
 'parents',
 'partner',
 'pre-nominals',
 'relatives',
 'religion',
 'resting_place',
 'resting_place_coordinates',
 'restingplace',
 'signature_type',
 'spouse',
 'spouse-type',
 'title',
 'website',
 'years_active',
 'yearsactive'}

In [4]:
class Infobox_Officeholder(Infobox):
    def __init__(self,title,property):
        self.type = "Infobox Officeholder"
        self.property = property
        Infobox.__init__(self,title)
    def clean_data(self):
        temp = {}
        for i in self.property :
            temp[i] = self.clean_text(i)
        temp['branch'] = self.clean_list("branch")
        temp['monarch3'] = self.clean_br("monarch3")
        temp['president3'] = self.clean_br("president3")
        temp['allegiance'] = self.clean_list2('allegiance')
        if len(temp['allegiance']) == 0 :
            temp['allegiance'] = self.clean_text('allegiance')
        temp['predecessor'] = self.clean_br('predecessor')
        temp['monarch'] = self.clean_br("monarch")
        temp['death_date'] = self.clean_date_template('death_date')
        temp['parents'] = self.clean_br('parents')
        temp['president2'] = self.clean_br("president2")
        temp['birth_date'] = self.clean_date_template('birth_date')
        temp['monarch2'] = self.clean_br("monarch2")
        temp['spouse'] = self.clean_br('spouse')
        temp['children'] = self.clean_br('children')
        temp['successor'] = self.clean_br('successor')
        temp['president'] = self.clean_br("president")
        if temp['battles'] == "" :
            temp['battles'] = self.clean_list('battles')
        #death_place,birth_place
        return temp

In [5]:
source = "infobox\\json\\v1Infobox_Officeholder37.json"
info_type = "Infobox Officeholder"
daftar_property = get_property(source,info_type)


85
Term_start5
birthname
spouse
governor
primeminister1
unit
order5
otherparty
succeeding
allegiance
image
imagesize
term_end3
president4
death_cause
predecessor4
predecessor3
term_start
successor4
order
predecessor
president2
term_start2
honorific-prefix
profession
president3
co-leader
term_start1
birth_date
successor
term_end2
primeminister
rank
successor2
title
term_end
serviceyears
monarch2
term_end1
signature
primeminister4
1
name
order2
term_start3
president1
awards
order4
office2
alma_mater
primeminister3
caption
death_date
monarch3
parents
monarch
governor3
office
predecessor1
successor3
successor5
religion
death_place
nationality
predecessor5
birth_place
battles
office1
primeminister2
office4
office5
branch
term_start4
relations
children
order3
office3
term_end5
term_start5
president
footnotes
successor1
predecessor2
term_end4
partner


In [6]:
infobox = json.load(open(source))
print(len(infobox))
daftar_infobox = []
for i in infobox :
    daftar_infobox.append(Infobox_Officeholder(i['entity'],daftar_property))
print(len(daftar_infobox))

17
 {{death date and age|1899|2|11|1854}}

['1899', '2', '11', '1854']
 {{death date and age|1998|9|11|1910|4|10}}

['1998', '9', '11', '1910', '4', '10']
 {{birth date|1910|4|10}}

['1910', '4', '10']
 {{death date and age|1978|3|4|1897|4|20}}

['1978', '3', '4', '1897', '4', '20']
 {{birth date|1897|4|20}}

['1897', '4', '20']
 {{death date and age|df=yes|1818|5|2|1762|10|21}}

['1818', '5', '2', '1762', '10', '21']
 {{birth date|df=yes|1762|10|21}}

['1762', '10', '21']
 {{Death date and age|1978|08|16|1888|03|07}} 

['1978', '08', '16', '1888', '03', '07']
 {{Birth date|1888|03|07}}

['1888', '03', '07']
 {{death date and age|1899|2|11|1854}}

['1899', '2', '11', '1854']
 {{Death date and age|1750|11|01|1705|08|08}}

['1750', '11', '01', '1705', '08', '08']
 {{Death date and age|1845|04|12|1779|5|24}}

['1845', '04', '12', '1779', '5', '24']
 {{birth date|1779|5|25}}

['1779', '5', '25']
 {{death date and age|df=y|1838|5|23|1762|10|12}}

['1838', '5', '23', '1762', '10', '12']
 {{b

In [8]:
key = 'birth_date'
for i in daftar_infobox :
    print(i.infobox.get(key,""))
    print("-----------------")
    print(i.clean_infobox[key])
#     for j,k in i.clean_infobox.items():
#         print(j,":",k)
    print("################")

 

-----------------

################
 [[1772]]

-----------------

################
 [[1854]]

-----------------

################
 {{birth date|1910|4|10}}

-----------------
10-April-1910
################
 {{birth date|1897|4|20}}

-----------------
20-April-1897
################
 {{birth date|df=yes|1762|10|21}}

-----------------
21-October-1762
################
 [[1860]] (perkiraan)

-----------------

################
 {{Birth date|1888|03|07}}

-----------------
07-March-1888
################
 [[1772]]

-----------------

################
 [[1854]]

-----------------

################
 [[8 Agustus]] [[1705]]

-----------------

################
 {{birth date|1779|5|25}}

-----------------
25-May-1779
################
 {{birth date|df=y|1762|10|12}}

-----------------
12-October-1762
################
 1760

-----------------

################
 {{birth date|1938|5|7}}

-----------------
7-May-1938
################
 {{Birth date|1917|7|1}}

-----------------
1-July-1917
#########

In [9]:
print(daftar_infobox[0].clean_infobox)

{'order': '', 'successor3': '', 'death_cause': '', 'primeminister': '', 'president4': '', 'president3': [], 'term_start1': '', 'successor5': '', 'office': '', 'governor3': '', 'caption': 'Raja Haji Fisabilillah', 'order3': '', 'term_start4': '', 'office2': '', 'religion': 'Islam', 'branch': [], 'monarch2': [], 'term_end4': '', 'office1': '', 'nationality': 'Indonesia', 'term_start2': '', 'Term_start5': '', 'death_place': '', 'monarch': [], 'unit': '', 'governor': '', 'parents': [''], 'honorific-prefix': '', 'imagesize': '200px', 'order4': '', 'order5': '', 'rank': '', 'battles': [], 'office5': '', 'predecessor5': '', 'term_end3': '', 'alma_mater': '', 'president': [], 'otherparty': '', 'term_start': '', 'term_end': '', 'successor': [''], 'successor2': '', 'primeminister1': '', 'awards': '', 'term_end1': '', 'monarch3': [], 'children': [], 'order2': '', 'birthname': '', 'successor4': '', 'relations': '', 'president1': '', 'co-leader': '', 'office4': '', 'birth_date': '', 'signature': ''

In [12]:
to_csv(daftar_infobox,"office_holder\\",'wikititle')

['Term_start5', 'birthname', 'governor', 'primeminister1', 'unit', 'order5', 'otherparty', 'succeeding', 'allegiance', 'image', 'imagesize', 'term_end3', 'president4', 'death_cause', 'predecessor4', 'predecessor3', 'term_start', 'successor4', 'order', 'term_start2', 'honorific-prefix', 'profession', 'co-leader', 'term_start1', 'birth_date', 'term_end2', 'primeminister', 'rank', 'successor2', 'title', 'term_end', 'serviceyears', 'term_end1', 'signature', 'primeminister4', '1', 'name', 'order2', 'term_start3', 'president1', 'awards', 'order4', 'office2', 'alma_mater', 'primeminister3', 'caption', 'death_date', 'governor3', 'office', 'predecessor1', 'successor3', 'successor5', 'religion', 'death_place', 'nationality', 'predecessor5', 'birth_place', 'office1', 'primeminister2', 'office4', 'office5', 'term_start4', 'relations', 'order3', 'office3', 'term_end5', 'term_start5', 'footnotes', 'successor1', 'predecessor2', 'term_end4', 'partner', 'wikititle', 'battles']


# infobox_military_person

In [13]:
class Infobox_Military_Person(Infobox):
    def __init__(self,title,property):
        self.type = "Infobox military person"
        self.property = property
        Infobox.__init__(self,title)
    def clean_data(self):
        temp = {}
        for i in self.property :
            temp[i] = self.clean_text(i)
        temp['awards'] = self.clean_list2('awards')
#         print(temp['awards'])
        if(len(temp['awards']) == 0):
            temp['awards'] = self.clean_br('awards')
#         print(temp['awards'])
        temp['spouse'] = self.clean_marriage('spouse')
        print(temp['spouse'])
        if(len(temp['spouse'])==0):
            temp['spouse'] = self.clean_br('spouse')
        print(temp['spouse'])
        temp['children'] = self.clean_br('children')
        temp['period'] = self.clean_date('period')
        temp['rank'] = self.clean_template('rank')
        temp['battles'] = self.clean_list2('battles')
        if(len(temp['battles']) == 0):
            temp['battles'] = self.clean_br('battles')
        temp['commands'] = self.clean_list2('commands')
        if(len(temp['commands']) == 0):
            temp['commands'] = self.clean_br('commands')
        temp['death_date'] = self.clean_date_template('death_date')
        temp['allegiance'] = self.clean_template("allegiance")
        temp['birth_date'] = self.clean_date_template('birth_date')
        temp['serviceyears'] = self.clean_date('serviceyears')
        return temp

In [14]:
source = "infobox\\json\\v1Infobox_military_person61.json"
info_type = "Infobox military person"
daftar_property = get_property(source,info_type)

32
width_style
native_name_lang
religion
serviceyears
spouse
death_place
laterwork
period
website
native_name
placeofburial_coordinates
birth_place
unit
battles
commands
servicenumber
name
branch
awards
allegiance
children
birth_date
caption
image_size
death_date
image
rank
serviceyears_label
nickname
battles_label
branch_label
birth_name


In [15]:
infobox = json.load(open(source))
print(len(infobox))
daftar_infobox = []
for i in infobox :
    daftar_infobox.append(Infobox_Military_Person(i['entity'],daftar_property))
print(len(daftar_infobox))

12
[]
[]
['I Gusti Ayu Made Geria', 'I Gusti Ayu Kompyang', 'Gusti Biyang Made Saji', 'Jero Sekar']
['']
['']
[]
[]
[]
['']
 {{Death date and age|1817|12|16|1783|06|08|df=yes}}

['1817', '12', '16', '1783', '06', '08']
 {{Birth date|1783|06|08|df=yes}}

['1783', '06', '08']
['\n']
['{{marriage|Agneta C. Roqué|1912|1925}}', '{{marriage|Anna Maria Bergman|1934}}']
['Agneta C. Roqué', 'Anna Maria Bergman']
['Agneta C. Roqué', 'Anna Maria Bergman']
['']
 {{Death date and age|df=yes|1968|1|15|1887|11|21}}

['1968', '1', '15', '1887', '11', '21']
 {{Birth date|df=yes|1887|11|21}}

['1887', '11', '21']
[' 1908', '1945\n']
[]
[]
[]
['']
 {{birth date|1886|10|11|df=y}}

['1886', '10', '11']
[' 1907', '1949\n']
[]
[]
[]
['']
 {{death date and age|1946|06|12|1879|08|08}}

['1946', '06', '12', '1879', '08', '08']
{{birth date|1879|08|08}}

['1879', '08', '08']
[' 1900', '1945\n']
[]
[]
[]
['']
{{death date and age|df=y|1968|10|04|1886|06|28}}

['1968', '10', '04', '1886', '06', '28']
{{birth date|

In [145]:
key = 'serviceyears'
for i in daftar_infobox :
    print(i.infobox.get(key,""))
    print("-----------------")
    print(i.clean_infobox[key])
#     for j,k in i.clean_infobox.items():
#         print(j,":",k)
    print("################")


-----------------
{'start': ''}
################


-----------------
{'start': ''}
################
 1908–1945

-----------------
{'start': '1908', 'end': '1945'}
################
 1907–1949

-----------------
{'start': '1907', 'end': '1949'}
################
 1900–1945

-----------------
{'start': '1900', 'end': '1945'}
################
 1907 - 1945

-----------------
{'start': '1907', 'end': '1945'}
################
1911–1944

-----------------
{'start': '1911', 'end': '1944'}
################

-----------------
{'start': ''}
################
 1906–1942
 
-----------------
{'start': '1906', 'end': '1942'}
################
 1822-1874

-----------------
{'start': '1822', 'end': '1874'}
################
1909–1945

-----------------
{'start': '1909', 'end': '1945'}
################
 1914-1945

-----------------
{'start': '1914', 'end': '1945'}
################


In [16]:
to_csv(daftar_infobox,"military_person\\",'wikititle')

['width_style', 'native_name_lang', 'religion', 'death_place', 'laterwork', 'website', 'native_name', 'placeofburial_coordinates', 'birth_place', 'unit', 'servicenumber', 'name', 'branch', 'allegiance', 'birth_date', 'caption', 'image_size', 'death_date', 'image', 'rank', 'serviceyears_label', 'nickname', 'battles_label', 'branch_label', 'birth_name', 'wikititle']
